# 1. Introducción

En el siguiente ejercicio se realizará el **producto escalar** entre dos vectores de igual dimensiones.

El algoritmo está basado en la función **dot** nivel 1 [1], de la biblioteca **BLAS** [2] que resuelve la ecuación:
<center>$res = \sum_{i-1}^{n} X_i * Y_i$</center>

La idea principal es mostrar la perfomance del funcionamiento de **estructuras de una dimensión** para gran cantidad de elementos.

Se utilizará Python [3] con las prestaciones brindadas por CUDA [4] para el uso de tecnología GPGPU en la plataforma Colab [5][6].

En este último aspecto, se implementó **una función kernel** que se encarga de la **multiplicación** de los componentes de los vectores con el objetivo de paralelizar esta tarea, para luego simplemente sumar los resultados parciales convencionalmente.

---
# 2. Armado del ambiente

### Importación de módulo **CUDA** y **bibliotecas**

In [ ]:
# Instalamos CUDA para Python
!pip install pycuda

# Importamos el driver y el compilador para CUDA
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Importamos bibliotecas estándares de Python
from datetime import datetime
import numpy
import math

---
# 3. Desarrollo

In [ ]:
#@title ### 3.1. Parámetros de ejecución {vertical-output: true}
#@markdown ---
#@markdown Cantidad de elementos de los arrays X e Y:
cant_elementos =   50000#@param {type: "integer"}
#@markdown ---

# Validamos la cantidad de elementos de los arrays
if not (type(cant_elementos) is int):
  raise TypeError("El parámetro de entrada debe ser un entero.") 
if cant_elementos <= 0:
  raise Exception("La cantidad de dimensiones de los arrays deben ser al menos 1.")

# Definición de función que transforma el tiempo en milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# El ejercicio podría fallar si no se importaron los recursos necesarios
# Es por eso que envolvemos el código en un bloque try, de esta forma
# se le indica al usuario cómo proceder
try: 
  # Capturamos el tiempo inicial
  tiempo_total = datetime.now()

  # CPU - Defino los vectores y el resultado en CPU
  x_cpu = numpy.random.randint(1, 10, cant_elementos)
  x_cpu = x_cpu.astype(numpy.int64())
  y_cpu = numpy.random.randint(1, 10, cant_elementos)
  y_cpu = y_cpu.astype(numpy.int64())

  # GPU - Reservo la memoria GPU en base a los vectores creados
  x_gpu = cuda.mem_alloc(x_cpu.nbytes)
  y_gpu = cuda.mem_alloc(y_cpu.nbytes)

  # GPU - Copio la memoria desde CPU a GPU
  cuda.memcpy_htod(x_gpu, x_cpu)
  cuda.memcpy_htod(y_gpu, y_cpu)

  # CPU - Defino la función kernel que multiplica los elementos
  module_mul = SourceModule("""
    __global__ void kernel_dot_mul(int n, int *X, int *Y) {
        
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      if(idx < n) {
        Y[idx] = X[idx] * Y[idx];
      }

    }
  """)

  # CPU - Obtenemos la función kernel creada
  kernel_mul = module_mul.get_function("kernel_dot_mul")

  # Capturamos el tiempo inicial del algoritmo y de GPU
  tiempo_algoritmo = datetime.now()
  tiempo_gpu = datetime.now()

  # GPU - Definimos la dimensión de threads y bloques
  dim_hilo = 16
  dim_bloque = numpy.int((cant_elementos + dim_hilo - 1) / dim_hilo)

  # GPU - Ejecuta el kernel para la multiplicación
  kernel_mul(numpy.int64(cant_elementos),
            x_gpu, y_gpu, 
            block=(dim_hilo, 1, 1), grid=(dim_bloque, 1, 1))

  # GPU - Copiamos el resultado desde la memoria GPU a CPU
  cuda.memcpy_dtoh(y_cpu, y_gpu)

  # Capturamos el tiempo total de GPU
  tiempo_gpu = datetime.now() - tiempo_gpu
  
  # Computamos los resultados parciales
  res_cpu = 0
  for e in range(0, cant_elementos):
    res_cpu += y_cpu[e]

  # Capturamos el tiempo total del algoritmo y de todo el ejercicio
  tiempo_algoritmo = datetime.now() - tiempo_algoritmo
  tiempo_total = datetime.now() - tiempo_total

  # Mostramos el resultado del proceso
  print("Cantidad de elementos en los arrays: ", cant_elementos)
  print("Dim. Thread x: ", dim_hilo, " - Dim. Bloque x:", dim_bloque, "\n")
  print("Tiempo GPU: ", tiempo_en_ms(tiempo_gpu), "[ms]")
  print("Tiempo DOT: ", tiempo_en_ms(tiempo_algoritmo), "[ms]")
  print("Tiempo total: ", tiempo_en_ms(tiempo_total), "[ms]\n")
  print("Resultado de DOT: ", res_cpu)
except:
  print("Ups! Algo salió mal, ¿Realizó la preparación previa del ambiente?")

---
# 4. Tabla de pasos de ejecución del programa


Procesador|Función|Detalle
---|---|---
CPU|pip install pycuda|Instala el módulo de CUDA para Python.
CPU|import|Importa los módulos para funcionar.
CPU|@param|Lectura del tamaño de vectores desde Colab.
CPU|datetime.now()|Toma el tiempo inicial del ejercicio.
CPU|numpy.random.randint|Inicializa los vectores X e Y con enteros aleatorios.
CPU|astype|Castea los vectores a un tipo de dato especificado.
CPU|numpy.empty_like|Retorna un array con la misma forma y tipo que el especificado.
**GPU**|cuda.mem_alloc()|Reserva la memoria en GPU.
**GPU**|cuda.memcpy_htod()|Copia la memoria desde el CPU al GPU.
CPU|SourceModule()|Define el código de las funciones kernel. 
CPU|module.get_function()|Obtenemos las funciones del kernel GPU.
CPU|datetime.now()|Toma el tiempo inicial del algoritmo y de GPU.
CPU|dim_tx/dim_bx|Calcula las dimensiones.
**GPU**|kernel_mul y kernel_sum|Ejecuta el kernel en GPU
**GPU**|cuda.memcpy_dtoh()|Copia el resultado desde memoria GPU a memoria CPU.
CPU|datetime.now()|Toma el tiempo total de GPU.
CPU|for(...)|Computa los resultados parciales de DOT
CPU|datetime.now()|Toma el tiempo total del algoritmo y del ejercicio.
CPU|print()|Informa los resultados.



---
# 5. Conclusiones

Se realizaron las siguientes pruebas:

Cantidad de elementos|Tiempo DOT [ms]|Tiempo Total [ms]|Relación tiempos [%] 
---|---|---|---
50K|12.71|16.12|78.46
500k|129.26|156.58|82.55
5M|1300.77|1524.78|85.30
50M|12443.11|14697.39|84.66

Podemos decir que:
*   A comparación de la versión CPU, el tiempo necesario que lleva el algoritmo bajó aproximadamente un 10% respecto al total, esto claramente se debe al **paralelismo** durante la realización de los cálculos intermedios. Antes, en CPU, absolutamente todos los cálculos se debían hacer uno atrás de otro, por lo que terminaba siendo razonable que casi todo el tiempo de ejecución se lo llevase el algortimo. Ahora, en GPU, los cálculos parciales de los componentes pueden ser paralelizados, simplificando así la tarea, y por ende, haciendo más rápido el algoritmo.

*   Si bien el tiempo total parece que **continua creciendo linealmente**, este se **redujo mucho** respecto a la versión CPU, con un aproximado del 70%. La linealidad continúa siendo razonable, ya que cuantos más elementos tengan los vectores, más se tardarán en crear y más tardarán en ser trasladados desde la memoria CPU a memoria GPU y viceversa. Sin embargo, como el tiempo total se ve ampliamente reducido con respecto a la versión CPU, podemos considerar que el problema adquiere la complejidad de una recta con menor pendiente.

Consideramos, entonces, que el uso de GPU en operaciones entre estructuras de una dimensión viene muy bien para esta clase de problemas ya que muchas veces todo se reduce a pequeños problemas independientes que pueden ser tomados por los distintos hilos. Como detalle adicional, comentamos que realizar la suma final a través de hilos no mejoraba significativamente el algoritmo, ya que había mucho overhead de por medio durante la sincronización de los mismos debido a que accedían al mismo espacio de memoria, cosa que no ocurría con la multiplicación de los componentes de los vectores. 

Una manera de mejorar el ejercicio podría ser el hecho de poder elegir distintas operaciones entre vectores conocidas en el álgebra lineal para poder comparar cuáles son más costosas (suma de vectores, cálculo de producto vectorial, etc.)

---
# 6. Bibliografia

[1] Función DOT de biblioteca BLAS: [Página Web](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-dot.html)

[2] Biblioteca BLAS: [BLAS](http://www.netlib.org/blas/)

[3] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[4] Documentación de CUDA para Python: [Página web](https://documen.tician.de/pycuda/index.html)

[5] Sintaxis Markdown Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

